This code is the original scraping script for Jan 1-March 31. It includes the cleaning stage, but this step will be redone after the .

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
from datetime import datetime, timedelta
import random

## Scrape the University of Iowa Crime Log

In [5]:
browser = webdriver.Chrome()
browser.maximize_window()

# open the Iowa City crime log page
url = 'https://safety.uiowa.edu/crime-log#accordion-item-2146-3'
browser.get(url)

# find the start date and end date input fields
start_date_input = browser.find_element(By.ID, 'StartDate')

# clear the start date and set it to the beginning of the year
start_date_input.clear()
start_date_input.send_keys('01-01')

# find search button and click
search_button = browser.find_element(By.XPATH, '//input[@type="button" and @value="Search"]')

browser.execute_script("arguments[0].click();", search_button)

# wait for results
time.sleep(5)

# find the table body
table_body = browser.find_element(By.ID, 'table-data')

# get all the rows in the table
rows = table_body.find_elements(By.TAG_NAME, 'tr')

# make an empty list to store data
crime_data = []

# loop through each row and extract the data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')

    # check if row has 6 columns
    if len(columns) == 6:
        case_number = columns[0].text
        classification = columns[1].text
        date_time_occurred = columns[2].text
        date_time_reported = columns[3].text
        location = columns[4].text
        disposition = columns[5].text

    # append the row data to the list
    crime_data.append({
        'Case Number': case_number,
        'Classification': classification,
        'Date/Time Occurred': date_time_occurred,
        'Date/Time Reported': date_time_reported,
        'Location': location,
        'Disposition': disposition
    })

# close browser
browser.quit()

# convert into Pandas DF
crime_df = pd.DataFrame(crime_data)

# convert to CSV file
crime_df.to_csv('uiowa_city_crime_data.csv', index=False)

# display
print(crime_df)



      Case Number                                  Classification  \
0    CRM-2025-284               Criminal Mischief, Simple Assault   
1    CRM-2025-283                           Liquor Law Violations   
2    CRM-2025-282                                        Trespass   
3    CRM-2025-286                           Liquor Law Violations   
4    CRM-2025-285                                        Burglary   
..            ...                                             ...   
315    CRM-2025-6                               Criminal Mischief   
316    CRM-2025-3                     Driving Under the Influence   
317    CRM-2025-7                               Indecent Exposure   
318    CRM-2025-4  Driving Under the Influence, Habitual Offender   
319  CSA-2025-153                                        Stalking   

                                Date/Time Occurred Date/Time Reported  \
0    Between 03/30/2025 14:05 and 03/30/2025 14:25   03/30/2025 14:29   
1                        

## Scrape the Weather

In [8]:
browser = webdriver.Chrome()
browser.maximize_window()

# open the Cedar Rapids Weather History page (Iowa City doesn't have a history option)
base_url = 'https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/'


# Set date range
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 31)
current_date = start_date

# Data storage
weather_data = []

while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = base_url + date_str
    print(f"Scraping {url}")
    browser.get(url)

    try:
        # Wait for weather table to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'mat-table')]"))
        )

        # Extract all table rows
        rows = browser.find_elements(By.XPATH, "//table[contains(@class, 'mat-table')]/tbody/tr")

        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            if columns:
                time_of_day = columns[0].text.strip()
                temp = columns[1].text.strip()
                dew_point = columns[2].text.strip()
                humidity = columns[3].text.strip()
                wind_direction = columns[4].text.strip()
                wind_speed = columns[5].text.strip()
                wind_gust = columns[6].text.strip()
                pressure = columns[7].text.strip()
                precip = columns[8].text.strip()
                condition = columns[9].text.strip()

                weather_data.append([date_str, time_of_day, temp, dew_point, humidity, wind_direction, wind_speed, wind_gust, pressure, precip, condition])

    except Exception as e:
        print(f"Failed to scrape {date_str}: {e}")

    current_date += timedelta(days=1)
    time.sleep(2) 

# close browser
browser.quit()

# Save to CSV
weather_df = pd.DataFrame(weather_data, columns=["Date", "Time", "Temperature", "Dew Point", "Humidity", "Wind", "Wind Speed", "Wind Gust", "Pressure", "Precip.", "Condition"])
weather_df.to_csv("cedarrapids_weather.csv", index=False)
print("Scraping completed. Data saved to wunderground_weather.csv")

# display
print(weather_df)
                

Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-01
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-02
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-03
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-04
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-05
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-06
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-07
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-08
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-09
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-01-10
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date

## Cleaning

In [21]:
# Split the crime 'Date/Time Occurred' into Date and Time
crime_df['Date/Time Reported'] = pd.to_datetime(crime_df['Date/Time Reported'], format = '%m/%d/%Y %H:%M')
crime_df['Date'] = crime_df['Date/Time Reported'].dt.date
crime_df['Time'] = crime_df['Date/Time Reported'].dt.strftime('%H:%M')

display(crime_df)

,Case Number,Classification,Date/Time Occurred,Date/Time Reported,Location,Disposition,Date,Time
0,CRM-2025-284,"Criminal Mischief, Simple Assault",Between 03/30/2025 14:05 and 03/30/2025 14:25,2025-03-30 14:29:00,General Hospital,Inactive,2025-03-30,14:29
1,CRM-2025-283,Liquor Law Violations,03/30/2025 02:55,2025-03-30 02:55:00,Slater Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:55
2,CRM-2025-282,Trespass,03/30/2025 02:40,2025-03-30 02:40:00,Kinnick Stadium,Closed - Suspect Issued Criminal Trespass Warning,2025-03-30,02:40
3,CRM-2025-286,Liquor Law Violations,03/30/2025 02:13,2025-03-30 02:13:00,Currier Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:13
4,CRM-2025-285,Burglary,Between 03/30/2025 01:00 and 03/30/2025 03:00,2025-03-30 13:55:00,Jacobson Athletic Building,Closed - Suspect Arrested,2025-03-30,13:55
...,...,...,...,...,...,...,...,...
315,CRM-2025-6,Criminal Mischief,01/01/2025 20:32,2025-01-01 20:32:00,North Campus Parking Ramp,Inactive,2025-01-01,20:32
316,CRM-2025-3,Driving Under the Influence,01/01/2025 19:12,2025-01-01 19:12:00,100 BLK W BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,19:12
317,CRM-2025-7,Indecent Exposure,01/01/2025 19:00,2025-01-02 06:14:00,General Hospital,Inactive,2025-01-02,06:14
318,CRM-2025-4,"Driving Under the Influence, Habitual Offender",01/01/2025 02:13,2025-01-01 02:13:00,200 BLK E BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,02:13


In [30]:
# convert 'Date' in weather data to DateTime format
weather_df['Date'] = pd.to_datetime(weather_df['Date'], format='%Y-%m-%d')
crime_df['Date'] = pd.to_datetime(crime_df['Date'])

print(weather_df)
print(crime_df)

           Date      Time Temperature Dew Point Humidity Wind Wind Speed  \
0    2025-01-01   1:52 AM       24 °F     20 °F     84 %  WNW     10 mph   
1    2025-01-01   2:52 AM       26 °F     21 °F     81 %    W     10 mph   
2    2025-01-01   3:52 AM       25 °F     21 °F     85 %    W     10 mph   
3    2025-01-01   4:52 AM       26 °F     22 °F     84 %    W     17 mph   
4    2025-01-01   5:52 AM       27 °F     23 °F     85 %  WNW     16 mph   
...         ...       ...         ...       ...      ...  ...        ...   
2519 2025-03-31   7:52 PM       43 °F     27 °F     53 %    N      9 mph   
2520 2025-03-31   8:52 PM       41 °F     26 °F     55 %    N      5 mph   
2521 2025-03-31   9:52 PM       39 °F     26 °F     60 %    N      6 mph   
2522 2025-03-31  10:52 PM       39 °F     26 °F     60 %  NNE      7 mph   
2523 2025-03-31  11:52 PM       37 °F     25 °F     62 %  NNE      6 mph   

     Wind Gust  Pressure Precip.      Condition Time Bucket  
0        0 mph  29.16 in 

In [28]:
# Create new column (time buckets) (this is for future analysis)
def assign_time_bucket(time):
    if pd.to_datetime(time).hour < 6:
        return 'Night'
    elif pd.to_datetime(time).hour < 12:
        return 'Morning'
    elif pd.to_datetime(time).hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

# Apply to both datasets
merged_df['Time Bucket'] = merged_df['Time'].apply(assign_time_bucket)
crime_df['Time Bucket'] = crime_df['Time'].apply(assign_time_bucket)

print(weather_df[['Time', 'Time Bucket']])

          Time Time Bucket
0      1:52 AM       Night
1      2:52 AM       Night
2      3:52 AM       Night
3      4:52 AM       Night
4      5:52 AM       Night
...        ...         ...
2519   7:52 PM     Evening
2520   8:52 PM     Evening
2521   9:52 PM     Evening
2522  10:52 PM     Evening
2523  11:52 PM     Evening

[2524 rows x 2 columns]


## Merge Datasets

In [37]:
'''
Weather data is recorded at specific time intervals (mostly on the hour),
so I'm going to match each crime's time to the nearest available weather record
for that day
'''
# convert 'Time' to datetime
crime_df['DateTime'] = pd.to_datetime(crime_df['Date'].astype(str) + ' ' + crime_df['Time'])
weather_df['DateTime'] = pd.to_datetime(weather_df['Date'].astype(str) + ' ' + weather_df['Time'])

# function to find closest available weather time for each crime entry
def find_nearest_time(crime_time, weather_times):
    return weather_times.iloc[(weather_times - crime_time).abs().argsort()[0]] # find closest time

# apply function to assign closest time from weather_df to each crime record
crime_df['Nearest Weather Time'] = crime_df['DateTime'].apply(lambda x: find_nearest_time(x, weather_df['DateTime']))

# convert back to time format
crime_df['Nearest Weather Time'] = crime_df['Nearest Weather Time'].dt.strftime('%H:%M')

# format 'Time' in weather_df
weather_df['Time'] = weather_df['DateTime'].dt.strftime('%H:%M')

# finally merge on date and nearest weather time
merged_df = crime_df.merge(weather_df, left_on=['Date', 'Nearest Weather Time'], right_on=['Date','Time'], how = 'left')

# display
display(merged_df)

,Case Number,Classification,Date/Time Occurred,Date/Time Reported,Location,Disposition,Date,Time_x,Time Bucket_x,DateTime_x,...,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Time Bucket_y,DateTime_y
0,CRM-2025-284,"Criminal Mischief, Simple Assault",Between 03/30/2025 14:05 and 03/30/2025 14:25,2025-03-30 14:29:00,General Hospital,Inactive,2025-03-30,14:29,Afternoon,2025-03-30 14:29:00,...,37 °F,80 %,WSW,23 mph,31 mph,28.71 in,0.0 in,Cloudy / Windy,Afternoon,2025-03-30 14:52:00
1,CRM-2025-283,Liquor Law Violations,03/30/2025 02:55,2025-03-30 02:55:00,Slater Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:55,Night,2025-03-30 02:55:00,...,57 °F,87 %,SSE,17 mph,26 mph,28.63 in,0.0 in,Mostly Cloudy,Night,2025-03-30 02:52:00
2,CRM-2025-282,Trespass,03/30/2025 02:40,2025-03-30 02:40:00,Kinnick Stadium,Closed - Suspect Issued Criminal Trespass Warning,2025-03-30,02:40,Night,2025-03-30 02:40:00,...,57 °F,87 %,SSE,17 mph,26 mph,28.63 in,0.0 in,Mostly Cloudy,Night,2025-03-30 02:52:00
3,CRM-2025-286,Liquor Law Violations,03/30/2025 02:13,2025-03-30 02:13:00,Currier Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:13,Night,2025-03-30 02:13:00,...,57 °F,84 %,SSE,17 mph,28 mph,28.65 in,0.0 in,Mostly Cloudy,Night,2025-03-30 01:52:00
4,CRM-2025-285,Burglary,Between 03/30/2025 01:00 and 03/30/2025 03:00,2025-03-30 13:55:00,Jacobson Athletic Building,Closed - Suspect Arrested,2025-03-30,13:55,Afternoon,2025-03-30 13:55:00,...,37 °F,76 %,WSW,22 mph,33 mph,28.71 in,0.0 in,Cloudy / Windy,Afternoon,2025-03-30 13:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,CRM-2025-6,Criminal Mischief,01/01/2025 20:32,2025-01-01 20:32:00,North Campus Parking Ramp,Inactive,2025-01-01,20:32,Evening,2025-01-01 20:32:00,...,20 °F,88 %,SW,3 mph,0 mph,29.28 in,0.0 in,Mostly Cloudy,Evening,2025-01-01 20:52:00
316,CRM-2025-3,Driving Under the Influence,01/01/2025 19:12,2025-01-01 19:12:00,100 BLK W BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,19:12,Evening,2025-01-01 19:12:00,...,19 °F,78 %,NW,8 mph,0 mph,29.27 in,0.0 in,Cloudy,Evening,2025-01-01 18:52:00
317,CRM-2025-7,Indecent Exposure,01/01/2025 19:00,2025-01-02 06:14:00,General Hospital,Inactive,2025-01-02,06:14,Morning,2025-01-02 06:14:00,...,17 °F,89 %,CALM,0 mph,0 mph,29.31 in,0.0 in,Fair,Night,2025-01-02 05:52:00
318,CRM-2025-4,"Driving Under the Influence, Habitual Offender",01/01/2025 02:13,2025-01-01 02:13:00,200 BLK E BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,02:13,Night,2025-01-01 02:13:00,...,20 °F,84 %,WNW,10 mph,0 mph,29.16 in,0.0 in,Fair,Night,2025-01-01 01:52:00


In [39]:
'''
messed up earlier by applying the time buckets to both datasets, 
undoing that here
'''

# drop duplicate columns
merged_df = merged_df.drop(columns=['Time_y', 'Time Bucket_y', 'DateTime_y'])

# fix the remaining columns' names
merged_df.rename(columns={'Time_x': 'Time', 'Time Bucket_x': 'Time Bucket', 'DateTime_x': 'DateTime'}, inplace=True)

# display cleaned up dataframe
display(merged_df)

,Case Number,Classification,Date/Time Occurred,Date/Time Reported,Location,Disposition,Date,Time,Time Bucket,DateTime,Nearest Weather Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,CRM-2025-284,"Criminal Mischief, Simple Assault",Between 03/30/2025 14:05 and 03/30/2025 14:25,2025-03-30 14:29:00,General Hospital,Inactive,2025-03-30,14:29,Afternoon,2025-03-30 14:29:00,14:52,43 °F,37 °F,80 %,WSW,23 mph,31 mph,28.71 in,0.0 in,Cloudy / Windy
1,CRM-2025-283,Liquor Law Violations,03/30/2025 02:55,2025-03-30 02:55:00,Slater Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:55,Night,2025-03-30 02:55:00,02:52,61 °F,57 °F,87 %,SSE,17 mph,26 mph,28.63 in,0.0 in,Mostly Cloudy
2,CRM-2025-282,Trespass,03/30/2025 02:40,2025-03-30 02:40:00,Kinnick Stadium,Closed - Suspect Issued Criminal Trespass Warning,2025-03-30,02:40,Night,2025-03-30 02:40:00,02:52,61 °F,57 °F,87 %,SSE,17 mph,26 mph,28.63 in,0.0 in,Mostly Cloudy
3,CRM-2025-286,Liquor Law Violations,03/30/2025 02:13,2025-03-30 02:13:00,Currier Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:13,Night,2025-03-30 02:13:00,01:52,62 °F,57 °F,84 %,SSE,17 mph,28 mph,28.65 in,0.0 in,Mostly Cloudy
4,CRM-2025-285,Burglary,Between 03/30/2025 01:00 and 03/30/2025 03:00,2025-03-30 13:55:00,Jacobson Athletic Building,Closed - Suspect Arrested,2025-03-30,13:55,Afternoon,2025-03-30 13:55:00,13:52,44 °F,37 °F,76 %,WSW,22 mph,33 mph,28.71 in,0.0 in,Cloudy / Windy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,CRM-2025-6,Criminal Mischief,01/01/2025 20:32,2025-01-01 20:32:00,North Campus Parking Ramp,Inactive,2025-01-01,20:32,Evening,2025-01-01 20:32:00,20:52,23 °F,20 °F,88 %,SW,3 mph,0 mph,29.28 in,0.0 in,Mostly Cloudy
316,CRM-2025-3,Driving Under the Influence,01/01/2025 19:12,2025-01-01 19:12:00,100 BLK W BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,19:12,Evening,2025-01-01 19:12:00,18:52,25 °F,19 °F,78 %,NW,8 mph,0 mph,29.27 in,0.0 in,Cloudy
317,CRM-2025-7,Indecent Exposure,01/01/2025 19:00,2025-01-02 06:14:00,General Hospital,Inactive,2025-01-02,06:14,Morning,2025-01-02 06:14:00,05:52,20 °F,17 °F,89 %,CALM,0 mph,0 mph,29.31 in,0.0 in,Fair
318,CRM-2025-4,"Driving Under the Influence, Habitual Offender",01/01/2025 02:13,2025-01-01 02:13:00,200 BLK E BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,02:13,Night,2025-01-01 02:13:00,01:52,24 °F,20 °F,84 %,WNW,10 mph,0 mph,29.16 in,0.0 in,Fair


In [41]:
# Realized there is more cleaning to be done (should've done before merging)
merged_df['Temperature'] = merged_df['Temperature'].str.extract('([0-9.]+)').astype(float)
merged_df['Dew Point'] = merged_df['Dew Point'].str.extract('([0-9.]+)').astype(float)
merged_df['Humidity'] = merged_df['Humidity'].str.extract('([0-9.]+)').astype(float)
merged_df['Wind Speed'] = merged_df['Wind Speed'].str.extract('([0-9.]+)').astype(float)
merged_df['Wind Gust'] = merged_df['Wind Gust'].str.extract('([0-9.]+)').astype(float)
merged_df['Pressure'] = merged_df['Pressure'].str.extract('([0-9.]+)').astype(float)
merged_df['Precip.'] = merged_df['Precip.'].str.extract('([0-9.]+)').astype(float)

# display the recleaned dataframe
display(merged_df)

,Case Number,Classification,Date/Time Occurred,Date/Time Reported,Location,Disposition,Date,Time,Time Bucket,DateTime,Nearest Weather Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,CRM-2025-284,"Criminal Mischief, Simple Assault",Between 03/30/2025 14:05 and 03/30/2025 14:25,2025-03-30 14:29:00,General Hospital,Inactive,2025-03-30,14:29,Afternoon,2025-03-30 14:29:00,14:52,43.0,37.0,80.0,WSW,23.0,31.0,28.71,0.0,Cloudy / Windy
1,CRM-2025-283,Liquor Law Violations,03/30/2025 02:55,2025-03-30 02:55:00,Slater Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:55,Night,2025-03-30 02:55:00,02:52,61.0,57.0,87.0,SSE,17.0,26.0,28.63,0.0,Mostly Cloudy
2,CRM-2025-282,Trespass,03/30/2025 02:40,2025-03-30 02:40:00,Kinnick Stadium,Closed - Suspect Issued Criminal Trespass Warning,2025-03-30,02:40,Night,2025-03-30 02:40:00,02:52,61.0,57.0,87.0,SSE,17.0,26.0,28.63,0.0,Mostly Cloudy
3,CRM-2025-286,Liquor Law Violations,03/30/2025 02:13,2025-03-30 02:13:00,Currier Hall,Closed - Referred to Office of Student Account...,2025-03-30,02:13,Night,2025-03-30 02:13:00,01:52,62.0,57.0,84.0,SSE,17.0,28.0,28.65,0.0,Mostly Cloudy
4,CRM-2025-285,Burglary,Between 03/30/2025 01:00 and 03/30/2025 03:00,2025-03-30 13:55:00,Jacobson Athletic Building,Closed - Suspect Arrested,2025-03-30,13:55,Afternoon,2025-03-30 13:55:00,13:52,44.0,37.0,76.0,WSW,22.0,33.0,28.71,0.0,Cloudy / Windy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,CRM-2025-6,Criminal Mischief,01/01/2025 20:32,2025-01-01 20:32:00,North Campus Parking Ramp,Inactive,2025-01-01,20:32,Evening,2025-01-01 20:32:00,20:52,23.0,20.0,88.0,SW,3.0,0.0,29.28,0.0,Mostly Cloudy
316,CRM-2025-3,Driving Under the Influence,01/01/2025 19:12,2025-01-01 19:12:00,100 BLK W BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,19:12,Evening,2025-01-01 19:12:00,18:52,25.0,19.0,78.0,NW,8.0,0.0,29.27,0.0,Cloudy
317,CRM-2025-7,Indecent Exposure,01/01/2025 19:00,2025-01-02 06:14:00,General Hospital,Inactive,2025-01-02,06:14,Morning,2025-01-02 06:14:00,05:52,20.0,17.0,89.0,CALM,0.0,0.0,29.31,0.0,Fair
318,CRM-2025-4,"Driving Under the Influence, Habitual Offender",01/01/2025 02:13,2025-01-01 02:13:00,200 BLK E BURLINGTON ST,Closed - Suspect Arrested,2025-01-01,02:13,Night,2025-01-01 02:13:00,01:52,24.0,20.0,84.0,WNW,10.0,0.0,29.16,0.0,Fair


In [42]:
# export the merged dataframe
merged_df.to_csv('merged_data.csv', index=False)